## Data 기본 탐색

### Data Import
* data import를 할 때, train_data와 test_data의 store_id가 중복되므로, 중복을 피하고자 test_data의 store_id에 다음과 같은 처리를 한다
* test_data.loc[:,'store_id'] = test_data.loc[:,'store_id'] + train_data.loc[:,'store_id'].max() + 1
* 나중에 Submit할 때는 store_id를 원상복구 해서 Submit하도록 한다
* 중복을 피하고자 하는 이유는, train_data와 test_data를 합쳐서 data로 놓고, EDA와 Data Preprocessing을 동시에 진행하고자 함이다.

In [1]:
import pandas as pd 
import numpy as np
import os, time, pickle

In [2]:
# Data Meta 정보 지정
data_path = "./data/"
train_filename = "train.csv"
test_filename = "test.csv"

In [3]:
train_data = pd.read_csv(data_path + train_filename)
test_data = pd.read_csv(data_path + test_filename)

In [4]:
# test.csv랑 train.csv의 store_id가 같아도 같은 상점인 것은 아님
test_data.loc[:,'store_id'] = test_data.loc[:,'store_id'] + train_data.loc[:,'store_id'].max() + 1

In [5]:
data = pd.concat([train_data, test_data], axis=0, ignore_index=True)

### Sequnce 길이 정하기
* test data의 store_id의 유니크한 값의 최소값 최대값을 보니 각각 44, 32959였다.
* LSTM, GRU 등의 장기기억유닛은 대체로 100~500이 좋다고 했다.
* 그러나 나의 경우, LSTM이나 GRU보다는 Conv1D가 좋았다 (결과적으로)
* 따라서, Sequence의 길이는 최소값인 44로 정해서 zero padding을 하지 않는 방향으로 정한다

In [6]:
test_data['store_id'].value_counts().describe()

count      200.000000
mean      2366.960000
std       4243.725649
min         44.000000
25%        392.750000
50%        982.000000
75%       2321.750000
max      32959.000000
Name: store_id, dtype: float64

In [7]:
time_length = test_data['store_id'].value_counts().describe()['min']
timelength = time_length
print(timelength)

44.0


### Data Type Converting
* 우선, date 컬럼과 time 컬럼의 경우, 합쳐서 datetime 컬럼으로 만든다
* 이를, pandas dataframe의 index로 사용할 경우, padnas timeseries의 기능을 사용할 수 있으니, 추후 index로 사용하리라고 마음 먹는다.
* installments 컬럼의 경우, 결측값이 있는데 이는 일시불이라고 컬럼 설명에 나와 있다.
* 따라서, installments 컬럼의 결측값은 0으로 채운다.

In [8]:
data['datetime'] = pd.DatetimeIndex(data['date']+' '+data['time'])

# fillna
data.loc[:,"installments"].fillna(0, inplace=True)

# drop columns
data.drop(['date', 'time'], axis='columns', inplace=True)

### Data Column Meta 지정

In [9]:
all_cols = list(data.columns)
key_cols = ['store_id']
index_cols = ['datetime']
feature_cols = list(set(all_cols) - set(key_cols+index_cols))

print(feature_cols)

['card_id', 'days_of_week', 'holyday', 'amount', 'installments']


## Data Sequence 마트 Wrangling

### store_image 만들기
* store_id에 대응하는 store_image를 만든다
* 하나의 store_image는 datetime을 index로 가지고, feature_cols = ['time', 'days_of_week', 'holyday', 'amount', 'card_id', 'date', 'installments'] 를 컬럼으로 가진다.
* store_image는 sotre_id가 Key인 파이썬 딕셔너리이다.

In [10]:
%%time

store_image = {}

for store_id in data['store_id'].unique():
    store_image[store_id] = pd.DataFrame(data.loc[data.store_id == store_id,feature_cols].values,
                                         index=data.loc[data.store_id == store_id,'datetime'].values, columns=feature_cols)

Wall time: 24.6 s


### Amount Column에 대해서 Daily Aggregation하기
* sotre_id에 대해서 day를 기준으로 aggregation한다.
* pandas timeseries의 resample 메서드가 이를 수행한다.
* Daily Aggregation한 결과는 daily_amount_dict에 저장하는데, key는 store_id이고 value는 day가 index이고 amount가 column인 pandas timeseries이다.
* Y도 동시에 정의하는데, Y의 정의는 100일 동안의 미래의 가맹점의 매출(=amount의 합)이므로, rolling(100)을 수행했을 때 Missing이 발생하면 Y가 없으므로 dropna로 날린다.
* Y는 column_amount_dict에 저장하는데, Key는 store_id이고 value는 y이다.

In [21]:
def df_check(df, value=0.0):
    df[df==False] = value
    df[df.isna()] = value
    return df

In [22]:
daily_amount_dict = {}
cumsum_amount_dict = {}
daily_amount_array_list = []
cumsum_amount_array_list = []

for store_id in data['store_id'].unique():
    # daily aggregation
    daily_amount = store_image[store_id]["amount"].resample('D').sum()
    daily_amount = df_check(daily_amount)
    daily_amount_array_list.append(daily_amount.values.reshape(len(daily_amount),1))
    daily_amount_dict[store_id] = daily_amount
    
    # Y: 각 상점의 마지막 매출 발생일 다음 날부터 100일 후까지 매출의 총합
    try:
        cumsum_amount = daily_amount.rolling(100).sum().dropna().shift(-100, freq='D')
        cumsum_amount_array_list.append(cumsum_amount.values.reshape(len(cumsum_amount),1))
        cumsum_amount_dict[store_id] = cumsum_amount
    except:
        print("cumsum_amount exception occured at {}".format(store_id))

cumsum_amount exception occured at 31
cumsum_amount exception occured at 42
cumsum_amount exception occured at 53
cumsum_amount exception occured at 61
cumsum_amount exception occured at 75
cumsum_amount exception occured at 93
cumsum_amount exception occured at 101
cumsum_amount exception occured at 102
cumsum_amount exception occured at 164
cumsum_amount exception occured at 183
cumsum_amount exception occured at 201
cumsum_amount exception occured at 203
cumsum_amount exception occured at 208
cumsum_amount exception occured at 229
cumsum_amount exception occured at 248
cumsum_amount exception occured at 254
cumsum_amount exception occured at 272
cumsum_amount exception occured at 295
cumsum_amount exception occured at 303
cumsum_amount exception occured at 308
cumsum_amount exception occured at 316
cumsum_amount exception occured at 321
cumsum_amount exception occured at 344
cumsum_amount exception occured at 350
cumsum_amount exception occured at 351
cumsum_amount exception occured

### Daily Amount Value와 Y Value를 StandardScaler로 표준화하기
* 나중에 Deep Learning 방법론을 사용할건데, Deep Learning 방법론은 Scaling에 굉장히 민감하다. 따라서 표준화를 진행한다.
* 앞서 정의했던, daily_amount_array_list와 cumsum_amount_array_list에 저장된 Value를 이용하여, Scaler를 학습시킨다.
* Scaler는 일반적으로 사용되는 sklearn.preprocessing.StandardScaler를 사용한다.

In [24]:
from sklearn.preprocessing import StandardScaler

daily_amount_values = np.vstack(daily_amount_array_list)
cumsum_amount_values = np.vstack(cumsum_amount_array_list)

daily_amount_scaler = StandardScaler()
cumsum_amount_scaler = StandardScaler()

daily_amount_scaler.fit(daily_amount_values)
cumsum_amount_scaler.fit(cumsum_amount_values)

StandardScaler()

### Scaling & time_length(=44) 만큼의 Sequence를 Feature로 Wranling한다
* daily_amount_scaler를 이용해서 daily_amount를 표준화
* time_length(=44) 동안의 Daily amount를 Columns으로서 사용
* 만약, 최근 44일 동안을 고려했을 때, 결측이 있는 경우 0으로 채움 (Zero padding)
* Test data에서 최소 44일 이상은 있는 것을 확인했으니, 43일이 지난 데이터가 있는 경우에만 Data로 인정
* Y도 역시 cumsum_amount_scaler를 이용해서 cumsum_amount를 표준화 (100일 지난 경우에만 가능)

In [25]:
from pandas.tseries.offsets import Day

seq_cols = ['amount_'+str(lag) for lag in np.arange(time_length-1, 0, -1)]
amounts_seq_dict = {}

for store_id in data['store_id'].unique():
    # Scaling
    daily_amount = daily_amount_dict[store_id]
    daily_amount_scaled = daily_amount_scaler.transform(daily_amount.values.reshape(len(daily_amount),1))
    daily_amount_scaled = pd.Series(daily_amount_scaled.reshape(len(daily_amount),), index=daily_amount.index)
    daily_amount_dict[store_id] = daily_amount_scaled
    
    # Sequence
    lag_amounts = []
    
    # time_length 동안의 Sequence 고려
    for lag in np.arange(time_length-1, 0, -1):
        lag_amount = daily_amount_scaled.shift(lag, freq='D')
        lag_amounts.append(pd.DataFrame(lag_amount.values, index=lag_amount.index, columns=['amount_'+str(lag)]))
        
    amounts_seq = pd.concat(lag_amounts+[daily_amount_scaled], axis='columns', join='outer')
    amounts_seq.columns = seq_cols + ['amount_0']
    
    # zero padding
    amounts_seq = amounts_seq.loc[daily_amount_scaled.index, :].fillna(0.0)
    # 적어도 43일 지났을 때부터 볼 것
    amounts_seq = amounts_seq.loc[daily_amount.index[0] + Day(43):,:]
    amounts_seq_dict[store_id] = amounts_seq
    
    # Y: 각 상점의 마지막 매출 발생일 다음 날부터 100일 후까지 매출의 총합
    try:
        cumsum_amount = cumsum_amount_dict[store_id]
        cumsum_amount_scaled = cumsum_amount_scaler.transform(cumsum_amount.values.reshape(len(cumsum_amount),1))
        cumsum_amount_dict[store_id] = pd.Series(cumsum_amount_scaled.reshape(len(cumsum_amount),), index=cumsum_amount.index)
    except:
        print("cumsum_amount exception occured at {}".format(store_id))

cumsum_amount exception occured at 31
cumsum_amount exception occured at 42
cumsum_amount exception occured at 53
cumsum_amount exception occured at 61
cumsum_amount exception occured at 75
cumsum_amount exception occured at 93
cumsum_amount exception occured at 101
cumsum_amount exception occured at 102
cumsum_amount exception occured at 164
cumsum_amount exception occured at 183
cumsum_amount exception occured at 201
cumsum_amount exception occured at 203
cumsum_amount exception occured at 208
cumsum_amount exception occured at 229
cumsum_amount exception occured at 248
cumsum_amount exception occured at 254
cumsum_amount exception occured at 272
cumsum_amount exception occured at 295
cumsum_amount exception occured at 303
cumsum_amount exception occured at 308
cumsum_amount exception occured at 316
cumsum_amount exception occured at 321
cumsum_amount exception occured at 344
cumsum_amount exception occured at 350
cumsum_amount exception occured at 351
cumsum_amount exception occured

## Data Non-Sequence(=Meta) 마트 Wrangling

### Meta - day 관련
* 중요한 정보는 현재 무슨 요일, 공휴일이냐가 아니라, 앞으로 100일 동안 공휴일이 얼마나 많이 들어있지? 등의 미래에 대한 정보 (Future information)
* 따라서 먼저, Futore information에 대한 정보를 Aggregation하기 전에, 먼저 무엇이 중요한 정보인지 판단한다.
* 주관적으로 생각했을 때, 중요한 정보는 아래와 같다.
    * 공휴일인가? (holyday)
    * 빨간 날인가? (빨간 날 = 일요일 + 공휴일)
    * 공휴일 전날인가?
    * 완전히 쉬는 날인가? (완전히 쉬는 날 = 토요일 + 일요일 + 공휴일)
    * 마음 놓고 유흥을 즐길 수 있는 날인가? (유흥 = 금요일 + 토요일 + 공휴일 전날)
    * 영업일인가? (근로자 영업일 = ~완전히 쉬는 날)

In [26]:
day_info_raw = pd.DataFrame(data.loc[:,["days_of_week", "holyday"]].values,
                            columns=["days_of_week", "holyday"],
                            index=data.loc[:,'datetime'].values)

In [27]:
day_info = day_info_raw.resample('D').max()

In [28]:
test = pd.get_dummies(day_info.index.weekday)

In [29]:
weekday_cols = ["week_"+str(i) for i in range(7)]

for i, col in enumerate(weekday_cols):
    weekday_info = pd.get_dummies(day_info.index.weekday)
    day_info[col] = weekday_info.iloc[:,i].values

In [30]:
# 공휴일 전날
day_info["ex-holyday"] = day_info["holyday"].shift(-1).fillna(0).astype(int).values

In [31]:
# 빨간 날 = 일요일 + 공휴일
day_info["red_day"] = day_info.loc[:,'week_6'] +  day_info.loc[:,'holyday'] - day_info.loc[:,'week_6']*day_info.loc[:,'holyday']

# 유흥 = 금요일 + 토요일 + 공휴일 전날
day_info["rest_day"] = day_info.loc[:,'week_4'] + day_info.loc[:,'week_5'] +  day_info.loc[:,'ex-holyday']\
- day_info.loc[:,'week_4']*day_info.loc[:,'week_5'] - day_info.loc[:,'week_4']*day_info.loc[:,'ex-holyday'] - day_info.loc[:,'week_5']*day_info.loc[:,'ex-holyday']\
+ day_info.loc[:,'week_4']*day_info.loc[:,'week_5']*day_info.loc[:,'ex-holyday']

# 완전히 쉬는 날 = 토요일 + 일요일 + 공휴일
day_info["rest_day"] = day_info.loc[:,'week_5'] + day_info.loc[:,'week_6'] +  day_info.loc[:,'holyday']\
- day_info.loc[:,'week_5']*day_info.loc[:,'week_6'] - day_info.loc[:,'week_5']*day_info.loc[:,'holyday'] - day_info.loc[:,'week_6']*day_info.loc[:,'holyday']\
+ day_info.loc[:,'week_5']*day_info.loc[:,'week_6']*day_info.loc[:,'holyday']

# 근로자 영업일 = ~완전히 쉬는 날

In [32]:
day_info.head()

,days_of_week,holyday,week_0,week_1,week_2,week_3,week_4,week_5,week_6,ex-holyday,red_day,rest_day
2016-08-01,0,0,1,0,0,0,0,0,0,0,0,0
2016-08-02,1,0,0,1,0,0,0,0,0,0,0,0
2016-08-03,2,0,0,0,1,0,0,0,0,0,0,0
2016-08-04,3,0,0,0,0,1,0,0,0,0,0,0
2016-08-05,4,0,0,0,0,0,1,0,0,0,0,0


### Meta -Future day infomation
* 중요한 정보는 현재 무슨 요일, 공휴일이냐가 아니라, 앞으로 100일 동안 공휴일이 얼마나 많이 들어있지? 등의 미래에 대한 정보 (Future information)
* 따라서 먼저, Futore information에 대한 정보를 Aggregation한다
* 주관적으로 생각했을 때, 중요한 정보는 아래와 같다.
    * 앞으로 100일 동안 예정된 예정된 월화수목금토일
    * 앞으로 100일 동안 예정된 빨간날
    * 앞으로 100일 동안 예정된 유흥날
    * 앞으로 100일 동안 예정된 쉬는날
*Aggregation한 후, StandardScaler로 표준화를 수행한다

In [33]:
# 앞으로 예정된 월화수목금토일 + 빨간날, 유흥날, 쉬는날 총 갯수
future_day_cols = list(day_info.columns)[3:]

future_day_info = day_info.loc[:,future_day_cols].rolling(100).sum().dropna().shift(-100, freq='D')

In [34]:
future_day_info.head()

,week_1,week_2,week_3,week_4,week_5,week_6,ex-holyday,red_day,rest_day
2016-07-31,15.0,14.0,14.0,14.0,14.0,14.0,6.0,19.0,33.0
2016-08-01,15.0,15.0,14.0,14.0,14.0,14.0,6.0,19.0,33.0
2016-08-02,14.0,15.0,15.0,14.0,14.0,14.0,6.0,19.0,33.0
2016-08-03,14.0,14.0,15.0,15.0,14.0,14.0,6.0,19.0,33.0
2016-08-04,14.0,14.0,14.0,15.0,15.0,14.0,6.0,19.0,34.0


In [35]:
day_scaler = StandardScaler()
future_day_info_scaled = day_scaler.fit_transform(future_day_info.values)
future_day_info_scaled = pd.DataFrame(future_day_info_scaled, columns=future_day_info.columns,
                                      index=future_day_info.index)

### Meta - Installments
* 카드 결제 시 일시납 또는 할부 몇개월을 한다는 것은 그 가맹점의 고유한 특징을 반영한다고 생각 (주관적)
* 따라서, Installments 컬럼에 대해서는 Day와 반대로 오히려 과거 Hisotry에 주목한다.
* 과거 총 카드 결제 중 일시납의 비율, 할부 X개월의 비율을 Feature로서 사용하기 위해, Aggregation을 수행한다.
* Aggregation 후에는, 역시 StandardScaler로 표준화를 수행한다.

In [36]:
installments_list = list(np.sort(data["installments"].unique().astype(int)))[1:]
installments_cols = ['installments_'+str(i) for i in installments_list]
print(installments_list)

[2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 18, 20, 22, 24, 36]


In [37]:
installment_dict = {}
installment_array_list = []

for store_id in data['store_id'].unique():
    # daily aggregation
    daily_amount = daily_amount_dict[store_id]
    daily_df = pd.DataFrame(daily_amount.values, columns=["amount"], index=daily_amount.index)
    
    for col in installments_cols:
        daily_df[col] = np.zeros(len(daily_df))
    
    for timeindex in daily_df.index:
        installment_image = store_image[store_id]["installments"]
        installment_history = installment_image[:timeindex].value_counts() / len(installment_image[:timeindex])
        for i in installment_history.index:
            daily_df.loc[timeindex, 'installments_'+str(int(i))] = installment_history[i]
    
    installment_dict[store_id] = daily_df.loc[:,installments_cols]
    installment_array_list.append(daily_df.loc[:,installments_cols].values)

In [38]:
installment_values = np.vstack(installment_array_list)
installment_scaler = StandardScaler()
installment_scaler.fit(installment_values)

StandardScaler()

In [39]:
for store_id in data['store_id'].unique():
    installment_df = installment_dict[store_id]
    installment_df.loc[:,:] = installment_scaler.transform(installment_df.values)
    installment_dict[store_id] = installment_df

## Data Join

### Data Join for AE (AutoEncoder)
* 위 2, 3번에서 말았던 Sequence data와 Day data, installments data를 Join한다.
* 이 때, AutoEncoder는 비지도 학습이므로, y(=cumsum_amount)는 필요 없다
* Output: X_AE

In [40]:
# For AE
X_AE_list = []

for store_id in data['store_id'].unique():
    amounts_seq = amounts_seq_dict[store_id]
    installment = installment_dict[store_id]
    X_seq_meta_df = pd.concat([amounts_seq, installment, future_day_info_scaled], axis='columns', join='inner')
    X_AE_list.append(X_seq_meta_df.values)

X_AE = np.vstack(X_AE_list)

### Data Join for Modeling
* 위 2, 3번에서 말았던 Sequence data와 Day data, installments data, y(=cumsum_amount)를 Join한다.
* 이 때, 지도 학습이므로, y(=cumsum_amount)도 필요한데, y값은 100일 지난 후까지의 데이터가 있어야 존재하므로, 이를 먼저 try 문으로 있는지 확인한다
* Output: X, y

In [41]:
X_list = []
y_list = []

for store_id in data['store_id'].unique():
    try:
        cumsum_amount_scaled = cumsum_amount_dict[store_id]
    except:
        pass
    else:
        amounts_seq = amounts_seq_dict[store_id]
        installment = installment_dict[store_id]
        index_intersection = cumsum_amount_scaled.index.intersection(amounts_seq.index)
        
        X_seq_df = amounts_seq.loc[index_intersection, :]
        X_seq_meta_df = pd.concat([X_seq_df, installment, future_day_info_scaled], axis='columns', join='inner')
        y_series = cumsum_amount_scaled[index_intersection]
        
        X_list.append(X_seq_meta_df.loc[index_intersection, :].values)
        y_list.append(y_series.values)

In [42]:
X = np.vstack(X_list)
y = np.concatenate(y_list, axis=0)

### Data Meta Define
* X의 Feature를 List로 저장한다

In [43]:
feature_cols = list(X_seq_meta_df.columns)
print(feature_cols)

['amount_43.0', 'amount_42.0', 'amount_41.0', 'amount_40.0', 'amount_39.0', 'amount_38.0', 'amount_37.0', 'amount_36.0', 'amount_35.0', 'amount_34.0', 'amount_33.0', 'amount_32.0', 'amount_31.0', 'amount_30.0', 'amount_29.0', 'amount_28.0', 'amount_27.0', 'amount_26.0', 'amount_25.0', 'amount_24.0', 'amount_23.0', 'amount_22.0', 'amount_21.0', 'amount_20.0', 'amount_19.0', 'amount_18.0', 'amount_17.0', 'amount_16.0', 'amount_15.0', 'amount_14.0', 'amount_13.0', 'amount_12.0', 'amount_11.0', 'amount_10.0', 'amount_9.0', 'amount_8.0', 'amount_7.0', 'amount_6.0', 'amount_5.0', 'amount_4.0', 'amount_3.0', 'amount_2.0', 'amount_1.0', 'amount_0', 'installments_2', 'installments_3', 'installments_4', 'installments_5', 'installments_6', 'installments_7', 'installments_8', 'installments_9', 'installments_10', 'installments_12', 'installments_15', 'installments_18', 'installments_20', 'installments_22', 'installments_24', 'installments_36', 'week_1', 'week_2', 'week_3', 'week_4', 'week_5', 'week

### Test Data
* test data는 AE와 비슷하게 y가 필요 없다.
* 하지만, 가장 최신의 데이터를 기준으로 미래 100을 예측하는 문제이므로, datetime 기준 가장 마지막에 있는 데이터를 사용한다
* 즉, pandas timeseries의 가장 마지막 데이터

In [44]:
X_test_list = []

for store_id in test_data['store_id'].unique():
    amounts_seq = amounts_seq_dict[store_id]
    installment = installment_dict[store_id]
    X_seq_df = amounts_seq.iloc[[-1], :]
    X_seq_meta_df = pd.concat([X_seq_df, installment, future_day_info_scaled], axis='columns', join='inner')
    X_test_list.append(X_seq_meta_df.iloc[[-1], :].values)

X_test = np.vstack(X_test_list)

## Data Save
* Data는 Python Pickle 파일로 저장한다.
* 저장하는 파일 목록은 다음과 같다.
    * (X, y) : Modeling Data
    * feautre_cols : Column Meta
    * X_test : Data for Scoring
    * columns_amount_scaler: Need for converting original value for y
    * X_AE : Data for Representation Learning

In [47]:
def write_pickle(data, path, file_name):
    with open("".join([path, '/', file_name, '.pkl']), 'wb') as f:
        pickle.dump(data, f)


def read_pickle(path, file_name):
    with open("".join([path, '/', file_name, '.pkl']), 'rb') as f:
        return pickle.load(f)

mart_path = './mart/'

In [48]:
os.makedirs(mart_path,exist_ok=True)

In [49]:
write_pickle((X, y), mart_path, '44_develop_data')
write_pickle(feature_cols, mart_path, '44_feature_cols')
write_pickle(X_test, mart_path, '44_X_test')
write_pickle(cumsum_amount_scaler, mart_path, '44_y_scaler')
write_pickle(X_AE, mart_path, '44_X_AE')